# PromptTemplare for single dynamic inputs

In [1]:
!pip install -q langchain==0.0.208 openai==0.27.8 python-dotenv

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_token = os.getenv("OPENAI_API_KEY")


In [ ]:
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_token,temperature=0)

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(input_variables=["query"],
                                 template=template)

# create LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# set the query you want to ask
input_data = {"query" : "What is the main advantage of quantum computing over classical computing?"}

response = chain.run(input_data)

print("Question:", input_data['query'])
print("Answer:", response)


Question: What is the main advantage of bo0ks 
Answer: I don't know.


In [8]:
# Construct a Fewshot prompt template
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """ 
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template= example_template
)

dynamic_promt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=['input'],
    example_separator='\n\n',
)

#Create the LLMChain for the dynamic prompt
chain = LLMChain(llm=llm, prompt=dynamic_promt)

# Run the LLMChain with input_data
input_data = {'input' : 'tiger'}
response = chain(input_data)

print(response)


{'input': 'tiger', 'text': 'tropical rainforests, grasslands, and mangrove swamps'}


In [10]:
print(example_prompt)
print(dynamic_promt)

input_variables=['animal', 'habitat'] output_parser=None partial_variables={} template=' \nAnimal: {animal}\nHabitat: {habitat}\n' template_format='f-string' validate_template=True
input_variables=['input'] output_parser=None partial_variables={} examples=[{'animal': 'lion', 'habitat': 'savanna'}, {'animal': 'polar bear', 'habitat': 'Arctic ice'}, {'animal': 'elephant', 'habitat': 'African grasslands'}] example_selector=None example_prompt=PromptTemplate(input_variables=['animal', 'habitat'], output_parser=None, partial_variables={}, template=' \nAnimal: {animal}\nHabitat: {habitat}\n', template_format='f-string', validate_template=True) suffix='Animal: {input}\nHabitat:' example_separator='\n\n' prefix='Identify the habitat of the given animal' template_format='f-string' validate_template=True


In [11]:
prompt_template.save("awesome_prompt.json")

In [12]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt("awesome_prompt.json")

In [15]:
examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template,
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

dynamic_prompt = FewShotPromptTemplate(examples=examples,
                                       example_prompt=example_prompt,
                                       suffix=suffix,
                                       prefix=prefix,
                                       input_variables=['query'],
                                       example_separator= '\n\n'
                                       )

chain = LLMChain(llm=llm, prompt=dynamic_prompt)

input_data = {'query' : 'How can I learn quantum computing?'}

response = chain.run(input_data)



In [16]:
print(response)

Just watch some YouTube tutorials and hope for the best.


In [17]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [33]:
# selecting the fewshot examples based on the max input prompt length.
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100
)

In [34]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator='\n'
)


In [35]:
dynamic_prompt_template

FewShotPromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, examples=None, example_selector=LengthBasedExampleSelector(examples=[{'query': 'How do you feel today?', 'answer': "As an AI, I don't have feelings, but I've got jokes!"}, {'query': 'What is the speed of light?', 'answer': 'Fast enough to make a round trip around Earth 7.5 times in one second!'}, {'query': 'What is a quantum computer?', 'answer': 'A magical box that harnesses the power of subatomic particles to solve complex problems.'}, {'query': 'Who invented the telephone?', 'answer': "Alexander Graham Bell, the original 'ringmaster'."}, {'query': 'What programming language is best for AI development?', 'answer': "Python, because it's the only snake that won't bite."}, {'query': 'What is the capital of France?', 'answer': 'Paris, the city of love and baguettes.'}, {'query': 'What is photosynthesis?', 'answer': "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'

In [36]:
chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)

input_data = {"query": "Who invented the iphone"}

respone = chain(input_data)

In [37]:
respone

{'query': 'Who invented the iphone',
 'text': 'Steve Jobs and his team at Apple, the geniuses behind the device that keeps us all glued to our screens.'}